## Simulation of <110> dumbbells diffusion in Fe-dilute Si alloy.
In this notebook, we use the onsager calculator we created in part 1 to compute transport coefficients for the dumbbell-mediated mechanism for dilute Si in Fe.

The data of the energies are taken from the database given along with the paper by Messina et. al. - https://doi.org/10.1016/j.actamat.2020.03.038

We can use the same state indexing that we used for the Fe-Cu alloy in part 2, since we use the same calculator.

In [1]:
import numpy as np

from scipy.constants import physical_constants
kB = physical_constants['Boltzmann constant in eV/K'][0]
from matplotlib import pyplot as plt

import pickle
import time

In [2]:
%%time
with open("Fe_Cu_Ni_Si.pkl","rb") as fl:
    onsagercalculator = pickle.load(fl)

CPU times: user 13min 17s, sys: 39 s, total: 13min 56s
Wall time: 13min 8s


In [3]:
# Identify the complex states from Messina et. al. and assign corresponding labels.
name_to_themo_star = {"1nnA":2, "1nnB":1, "2nnA":3, "2nnB":4,"3nnA":7,"3nnB":5,"3nnC":6,
       "4nnA":8,"4nnB":11,"4nnC":10, "5nnA":13, "5nnB":12}

In [4]:
# sorting out the jumps with the nomenclatures for which solute-dumbbell interactions are taken into account
# in the paper by Messina et. al.
jmpdict = {"1nnA_2nnA":[], "1nnA_2nnB":[], "1nnA_3nnB":[], "1nnA_3nnC":[], "1nnB_2nnB":[], "1nnB_3nnB":[],
          "1nnB_5nnB":[], "2nnA_4nnC":[], "2nnB_4nnB":[], "2nnB_4nnC":[]}

# Now identify the jumps and put them into the dictionaries
for jlistind, jlist in enumerate(onsagercalculator.jnet1):
    
    # get the first jump of the symmetry-unique omega1 jump list
    jmp = jlist[0]
    state1 = jmp.state1
    state2 = jmp.state2
    # if rigid jump, then continue - solute-dumbbell interactions are not taken into
    # account for rigid jumps.
    if jmp.state1.db.iorind == jmp.state2.db.iorind:
        continue
    
    # get the symmetry groups ("stars") of the initial and final states
    star1 = onsagercalculator.kinetic.complexIndexdict[state1][1]
    star2 = onsagercalculator.kinetic.complexIndexdict[state2][1]
    
    # next, we see if the these stars are in the thermodynamic shell, and if the corresponding
    # jump is in our named jump dictionary above.
    if star1 in onsagercalculator.thermo2kin and star2 in onsagercalculator.thermo2kin:
        thermo_star1 = onsagercalculator.thermo.complexIndexdict[state1][1]
        thermo_star2 = onsagercalculator.thermo.complexIndexdict[state2][1]
        name1 = ""
        name2 = ""
        
        #Now see which stars the states belong to
        star1found = False
        count1 = 0
        star2found = False
        count2 = 0
        for (key, value) in name_to_themo_star.items():
            if thermo_star1==value:
                star1found = True
                count1 += 1
                name1 = key
            if thermo_star2==value:
                star2found = True
                count2 += 1
                name2 = key
        
        # just to ensure we don't have any multiple counting.
        if count1>1:
            print(thermo_star1)
        if count2>1:
            print(thermo_star2)
        
        # Now concatenate names
        jname = name1+"_"+name2
        jnameRev = name2+"_"+name1
        try:
            jmpdict[jname].append(jlistind)
        except:
            try:
                # maybe the jump we have is the reverse of what we stored as the label in the dictionary?
                jmpdict[jnamerev].append(jlistind)
            
            except:    
                continue

# check to see all necessary jumps have been found - only one symmetry group for each jump
# must be present
jmpdict

{'1nnA_2nnA': [1],
 '1nnA_2nnB': [6],
 '1nnA_3nnB': [5],
 '1nnA_3nnC': [3],
 '1nnB_2nnB': [4],
 '1nnB_3nnB': [7],
 '1nnB_5nnB': [2],
 '2nnA_4nnC': [8],
 '2nnB_4nnB': [9],
 '2nnB_4nnC': [10]}

In [5]:
# Next, supercell energies of the identified states
E_f_pdb = 4.081701163
name_to_en = {"1nnA":-2079.59136864, "1nnB":-2080.03839805, "2nnA":-2079.70049869, "2nnB":-2079.80935148, 
              "3nnA":-2079.75574574, "3nnB":-2079.74713030, "3nnC":-2079.69119146, "4nnA":-2079.75368298, 
              "4nnB":-2079.84525470, "4nnC":-2079.78250279, "5nnA":-2079.76427954, "5nnB":-2079.80243840}

In [6]:
E_sup_pdb = -2081.44451396
E_sup_solute = -2075.53747594 
E_bulk = -2077.21734574  #E_bulk is the same as E_ref
name_to_Ef = {}
for (key, E_IB) in name_to_en.items():
    # get the binding energy first
    Eb = -E_IB + E_sup_pdb + E_sup_solute - E_bulk
    # Next, get the formation energy (relative to solute formation energy)
    name_to_Ef[key] = E_f_pdb - Eb

In [7]:
# The complex energies are set. Now, we set the mixed dumbbell energies
E_b_mdb = 2079.76221064 + E_sup_pdb + E_sup_solute - E_bulk
E_f_mdb = E_f_pdb - E_b_mdb

In [8]:
# Next, the supercell energies of the transition states for the omega1 jumps in the database

# J_2_KRA_dumbbell_dumbbell       -2081.1091
# J_1_KRA_dumbbell_dumbbell       -2080.8336
# J_2_KRA_dumbbell_dumbbell       -2080.6599
# J_2_KRA_0nnA_0nnA               -2079.1947
# J_2_KRA_0nnA_0nnA               -2079.1872
# J_2_KRA_0nnA_1nnB               -2079.6930638

Jname_2_TS_en = {"1nnA_2nnA": -2079.2970, "1nnA_2nnB": -2079.3621, "1nnA_3nnB": -2079.3136,
                 "1nnA_3nnC": -2079.2797, "1nnB_2nnB": -2079.5994, "1nnB_3nnB": -2079.5636,
                 "1nnB_5nnB": -2079.5626, "2nnA_4nnC": -2079.3122, "2nnB_4nnB": -2079.5204, 
                 "2nnB_4nnC": -2079.4476}

In [10]:
# Now, we have to find the TS formation energies.
Jname_2_ef_ts = {}
for (key, E_IB) in Jname_2_TS_en.items():
    Eb = -E_IB + E_sup_pdb + E_sup_solute - E_bulk
    # Next, get the formation energy (relative to solute formation energy)
    Jname_2_ef_ts[key] = E_f_pdb - Eb

In [11]:
# Then compute the migration energies
Jname_2_mig = {}
for (key, TS_en) in Jname_2_ef_ts.items():
    initstar = key[:4]
    Jname_2_mig[key] = TS_en - name_to_Ef[initstar]

In [12]:
# omega2 and omega43 roto-translational jumps
E_IB_43, E_IB_2 = -2079.69306384,  -2079.1947
Eb_43, Eb_2 = -E_IB_43 + E_sup_pdb + E_sup_solute - E_bulk, -E_IB_2 + E_sup_pdb + E_sup_solute - E_bulk 
# Next, get the formation energy (relative to solute formation energy)
ef_ts_43 = E_f_pdb - Eb_43
ef_ts_2 = E_f_pdb - Eb_2

In [13]:
# omega2 rigid translation
E_IB_2_rigid = -2079.1872
Eb_2_rigid = -E_IB_2_rigid + E_sup_pdb + E_sup_solute - E_bulk
ef_ts_2_rigid = E_f_pdb - Eb_2_rigid

## Si calculations

In [14]:
# Jump rates and energy barriers set. Now, let's set the calculations up.
vu0 = 4.4447 # attempt frequencies of pure dumbbell jumps.
vu2 = 19.2684 # attempt frequencies of mixed dumbbell jumps.
Dconv=1e-2 # to get cm^2/s units from nm^2*THz

# pre-factors and formation energies for pure dumbbell jumps
predb0, enedb0 = np.ones(1)*np.exp(0.050), np.array([E_f_pdb])

# Here on, pre-factors are going to 1.0
# We'll measure every formation energy relative to the solute formation energy.
# so we set solute formation energy to zero, and (as per data) pre-factor to 1.
preS, eneS = np.ones(1), np.array([0.0])

# Next, interaction or the excess energies and pre-factors for solutes and dumbbells.
preSdb, eneSdb = np.ones(onsagercalculator.thermo.mixedstartindex), \
                 np.zeros(onsagercalculator.thermo.mixedstartindex)
# Now, we go over the necessary stars and assign interaction energies
for (key, index) in name_to_themo_star.items():
    eneSdb[index] = name_to_Ef[key] - E_f_pdb

predb2, enedb2 = np.ones(1), np.array([E_f_mdb])

# Transition state energies - For omega0, omega2 and omega43, the first type is the Johnson jump,
# and the second one is the Rigid jump.

# Omega0 TS eneriges
preT0, eneT0 = Dconv*vu0*np.ones(1), np.array([E_f_pdb+0.335115123, E_f_pdb + 0.61091396, E_f_pdb+0.784315123])

# Omega2 TS energies
Nj2 = len(onsagercalculator.jnet2)
preT2, eneT2 = Dconv*vu2*np.ones(Nj2), np.array([ef_ts_2, ef_ts_2_rigid])

# Omega43 TS energies
preT43, eneT43 = Dconv*vu0*np.ones(1), np.array([ef_ts_43])

# Omega1 TS energies
preT1 = Dconv*vu0*np.ones(len(onsagercalculator.jnet1))
eneT1 = np.array([eneT0[i] for i in onsagercalculator.om1types])
# Now, we go over the jumps that are provided and make the necessary changes
for (key, index) in jmpdict.items():
    eneT1[index] = Jname_2_ef_ts[key]
eneT1[0] = 0.0
# print(eneT1)

In [15]:
# Now we calculate the transport coefficients for all temperatures given in the data set

from tqdm import tqdm
temp = np.arange(200, 4001, 10)

diff_aa_Si = np.zeros(len(temp))
diff_ab_Si = np.zeros(len(temp))
diff_bb = np.zeros(len(temp))
diff_bb_non_loc = np.zeros(len(temp))

for i in tqdm(range(len(temp)), position=0, leave=True, ncols=65):
    T = temp[i]
    kT = kB*T
    bFdb0, bFdb2, bFS, bFSdb, bFT0, bFT1, bFT2, bFT3, bFT4 = \
        onsagercalculator.preene2betafree(kT, predb0, enedb0, preS, eneS, preSdb, eneSdb, predb2, enedb2,
                                               preT0, eneT0, preT2, eneT2, preT1, eneT1, preT43, eneT43)

    # get the transport coefficients
    # the uncorrelated (L_uc) and correlated (L_c) parts will be summed to get the total
    # transport coefficients.
    # "a" stands for solute, "b" for solvent (Fe)
    L0bb, (L_uc_aa,L_c_aa), (L_uc_bb,L_c_bb), (L_uc_ab,L_c_ab)=\
    onsagercalculator.L_ij(bFdb0, bFT0, bFdb2, bFT2, bFS, bFSdb, bFT1, bFT3, bFT4)
    
    L_aa = L_uc_aa + L_c_aa
    L_bb = L_uc_bb + L_c_bb
    L_ab = L_uc_ab + L_c_ab
    
    diff_aa_Si[i] = L_aa[0][0]
    diff_ab_Si[i] = L_ab[0][0]
    diff_bb[i] = L_bb[0][0]
    diff_bb_non_loc[i] = L0bb[0][0]

100%|██████████████████████████| 381/381 [43:10<00:00,  6.80s/it]


In [16]:
# Now let's do the infinite temeperature limit
kT = np.inf
bFdb0, bFdb2, bFS, bFSdb, bFT0, bFT1, bFT2, bFT3, bFT4 = \
    onsagercalculator.preene2betafree(kT, predb0, enedb0, preS, eneS, preSdb, eneSdb, predb2, enedb2,
                                           preT0, eneT0, preT2, eneT2, preT1, eneT1, preT43, eneT43)
#     bFdicts[i] = [bFdb0, bFdb2, bFS, bFSdb, bFT0, bFT1, bFT2, bFT3, bFT4]
# get the probabilities and other data from L_ij
L0bb, (L_uc_aa,L_c_aa), (L_uc_bb,L_c_bb), (L_uc_ab,L_c_ab)=\
onsagercalculator.L_ij(bFdb0, bFT0, bFdb2, bFT2, bFS, bFSdb, bFT1, bFT3, bFT4)

L_aa = L_uc_aa + L_c_aa
L_bb = L_uc_bb + L_c_bb
L_ab = L_uc_ab + L_c_ab

drag_inf = L_ab[0][0]/L_aa[0][0]
drag_inf

1.4821923711250593

In [18]:
import h5py
with h5py.File("Si_data.h5","w") as fl:
    fl.create_dataset("diff_aa", data=diff_aa_Si)
    fl.create_dataset("diff_ab", data=diff_ab_Si)
    fl.create_dataset("diff_bb_nl", data=diff_bb_non_loc)
    fl.create_dataset("diff_bb", data=diff_bb)
    fl.create_dataset("drag_inf", data=drag_inf)
    fl.create_dataset("Temp", data=temp)